In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e11/sample_submission.csv
/kaggle/input/playground-series-s5e11/train.csv
/kaggle/input/playground-series-s5e11/test.csv


In [17]:
# Reading the training data as a pandas dataframe
train_df = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv')

In [18]:
# Getting a view of the dataframe
print(f'List of columns in the dataframe {train_df.columns}')
train_df.head()

List of columns in the dataframe Index(['id', 'annual_income', 'debt_to_income_ratio', 'credit_score',
       'loan_amount', 'interest_rate', 'gender', 'marital_status',
       'education_level', 'employment_status', 'loan_purpose',
       'grade_subgrade', 'loan_paid_back'],
      dtype='object')


,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [19]:
# Splitting the columns according to its type
# Dropping the id column since it won't add any relevance to predict output

unused_col = ['id']

numeric_col = ['annual_income', 'debt_to_income_ratio', 'credit_score', 
               'loan_amount', 'interest_rate']

ohe_categorical_col = ['gender', 'marital_status', 'education_level', 'employment_status', 
                       'loan_purpose']

hash_categorical_col = ['grade_subgrade']

output_col = ['loan_paid_back']

In [20]:
# Checking the null values
train_df.info()

# The data looks clean with no need of cleaning required

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    593994 non-null  int64  
 1   annual_income         593994 non-null  float64
 2   debt_to_income_ratio  593994 non-null  float64
 3   credit_score          593994 non-null  int64  
 4   loan_amount           593994 non-null  float64
 5   interest_rate         593994 non-null  float64
 6   gender                593994 non-null  object 
 7   marital_status        593994 non-null  object 
 8   education_level       593994 non-null  object 
 9   employment_status     593994 non-null  object 
 10  loan_purpose          593994 non-null  object 
 11  grade_subgrade        593994 non-null  object 
 12  loan_paid_back        593994 non-null  float64
dtypes: float64(5), int64(2), object(6)
memory usage: 58.9+ MB


In [21]:
# Obtaining the descriptive stats of different available columns
train_df.describe()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,loan_paid_back
count,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000
mean,296996.500000,48212.202976,0.120696,680.916009,15020.297629,12.356345,0.798820
std,171471.442235,26711.942078,0.068573,55.424956,6926.530568,2.008959,0.400883
min,0.000000,6002.430000,0.011000,395.000000,500.090000,3.200000,0.000000
25%,148498.250000,27934.400000,0.072000,646.000000,10279.620000,10.990000,1.000000
50%,296996.500000,46557.680000,0.096000,682.000000,15000.220000,12.370000,1.000000
75%,445494.750000,60981.320000,0.156000,719.000000,18858.580000,13.680000,1.000000
max,593993.000000,393381.740000,0.627000,849.000000,48959.950000,20.990000,1.000000


In [22]:
# Checking the data for different type of loan purposes
train_df['loan_paid_back'].value_counts()

loan_paid_back
1.0    474494
0.0    119500
Name: count, dtype: int64

In [25]:
train_df = train_df.drop(unused_col, axis=1)

In [26]:
train_df

,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
593989,23004.26,0.152,703,20958.37,10.92,Female,Single,High School,Employed,Business,C3,1.0
593990,35289.43,0.105,559,3257.24,14.62,Male,Single,Bachelor's,Employed,Debt consolidation,F5,1.0
593991,47112.64,0.072,675,929.27,14.13,Female,Married,Bachelor's,Employed,Debt consolidation,C1,1.0
593992,76748.44,0.067,740,16290.40,9.87,Male,Single,Bachelor's,Employed,Debt consolidation,B2,1.0


In [27]:
X_col = [x for x in train_df if x != 'loan_paid_back']
y_col = 'loan_paid_back'

In [28]:
X_col

['annual_income',
 'debt_to_income_ratio',
 'credit_score',
 'loan_amount',
 'interest_rate',
 'gender',
 'marital_status',
 'education_level',
 'employment_status',
 'loan_purpose',
 'grade_subgrade']

In [29]:
from sklearn.model_selection import train_test_split, StratifiedKFold

X = train_df[X_col]
y = train_df[y_col]

# stratified train/test split (preserve class proportions)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [38]:
class SklearnFeatureHasher_stringtoken(BaseEstimator, TransformerMixin):
    """
    Wrap FeatureHasher with input_type='string' but ensure each sample is an iterable
    of strings (here: a single-token list [value]).
    """
    def __init__(self, n_features=32):
        self.n_features = n_features
        self.input_type = 'string'

    def fit(self, X, y=None):
        self._hasher = FeatureHasher(n_features=self.n_features, input_type=self.input_type)
        return self

    def transform(self, X):
        # Accept Series, DataFrame (single column), 1D array, or list
        if isinstance(X, (pd.Series, pd.DataFrame)):
            arr = X.values.ravel()
        else:
            arr = np.asarray(X).ravel()

        # Convert NaNs to string (or whatever sentinel you prefer)
        arr = np.where(pd.isna(arr), '___nan___', arr).astype(str)

        # IMPORTANT: FeatureHasher(input_type='string') expects each sample to be an iterable
        # of strings (e.g. tokens). Wrap each string into a single-token list:
        samples = [[val] for val in arr]

        hashed = self._hasher.transform(samples)   # returns sparse matrix
        if not sparse.isspmatrix_csr(hashed):
            hashed = hashed.tocsr()
        return hashed

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression  # example estimator
from sklearn.feature_extraction import FeatureHasher

# 3) Create ColumnTransformer - scale numeric cols, passthrough other columns
preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False), ohe_categorical_col),
        ('hash_cat', SklearnFeatureHasher_stringtoken(n_features=32), hash_categorical_col),
        ('num', StandardScaler(), numeric_col)
    ]
)

# 6) Example: plug into a pipeline with an estimator (keeps scaling inside CV correctly)
pipeline = Pipeline([
    ('pre', preprocessor),
    ('clf', LogisticRegression(max_iter=2000))
])

pipeline.fit(X_train, y_train)
print("Train score:", pipeline.score(X_train, y_train))
print("Test score:", pipeline.score(X_test, y_test))

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Train score: 0.900642893969844
Test score: 0.9005968063704239


In [40]:
test_df = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')

In [42]:
test_df = test_df.drop(unused_col, axis=1)
test_df

,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
0,28781.05,0.049,626,11461.42,14.73,Female,Single,High School,Employed,Other,D5
1,46626.39,0.093,732,15492.25,12.85,Female,Married,Master's,Employed,Other,C1
2,54954.89,0.367,611,3796.41,13.29,Male,Single,Bachelor's,Employed,Debt consolidation,D1
3,25644.63,0.110,671,6574.30,9.57,Female,Single,Bachelor's,Employed,Debt consolidation,C3
4,25169.64,0.081,688,17696.89,12.80,Female,Married,PhD,Employed,Business,C1
...,...,...,...,...,...,...,...,...,...,...,...
254564,92835.97,0.068,744,29704.00,13.48,Female,Single,Bachelor's,Employed,Debt consolidation,B2
254565,48846.47,0.091,634,20284.33,9.58,Female,Married,High School,Employed,Debt consolidation,D4
254566,20668.52,0.096,718,26387.55,9.00,Male,Single,Master's,Employed,Debt consolidation,C4
254567,34105.09,0.094,739,11107.36,9.81,Male,Single,Bachelor's,Employed,Business,C2


In [44]:
# get probability for positive class
probs = pipeline.predict_proba(test_df)[:, 1]

In [45]:
probs

array([0.93240959, 0.97947142, 0.14300639, ..., 0.97033105, 0.98124519,
       0.83291448])

In [46]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')
sub[sub.columns[1]] = probs

In [47]:
sub

,id,loan_paid_back
0,593994,0.932410
1,593995,0.979471
2,593996,0.143006
3,593997,0.918621
4,593998,0.967336
...,...,...
254564,848558,0.990710
254565,848559,0.895398
254566,848560,0.970331
254567,848561,0.981245


In [48]:
sub.to_csv('submission_feature_hashed.csv', index=False)